# Importing Libraries

In [21]:
import torch
import torchvision
import torch.nn as nn 
from torchvision.transforms import transforms
import glob
import cv2
from PIL import Image
import matplotlib.pyplot as plt
device = torch.cuda.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyperparameters

In [22]:
num_epochs = 12
lr = 0.01
batch_size = 4

# Downloading the CIFAR-10 Dataset

In [24]:
my_transform = transforms.ToTensor()
train_dataset = torchvision.datasets.CIFAR10(root='./cifar', train=True, download=True, transform=my_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./cifar', train = False, transform=my_transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
n_samples = len(train_dataset)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./cifar
    Split: Test
    StandardTransform
Transform: ToTensor()


# Random mask generator

In [ ]:
class 